In [1]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())

PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
cuDNN version: 90100


In [ ]:
!git clone https://github.com/PDA-PRO/COCO_AI.git
!git clone https://github.com/PDA-PRO/COCO_Back_End.git

Cloning into 'COCO_AI'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 316 (delta 49), reused 70 (delta 29), pack-reused 212 (from 1)
Receiving objects: 100% (316/316), 4.20 MiB | 11.45 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [ ]:
!pip install tree-sitter-python
!pip install transformers
!apt install unzip -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.2/544.2 kB 20.1 MB/s eta 0:00:00
Found existing installation: tree-sitter 0.22.3
Uninstalling tree-sitter-0.22.3:
  Successfully uninstalled tree-sitter-0.22.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.9/559.9 kB 15.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
codebleu 0.7.0 requires tree-sitter<0.23.0,>=0.22.0, but you have tree-sitter 0.23.0 which is incompatible.


## 알고리즘 문제 설명, 테스트 케이스 준비

In [ ]:
import os
import zipfile
import subprocess

# 1. 압축 해제 대상 경로
base_zip_path = "COCO_Back_End/base_task_set/base_task_set.zip"  # 분할압축의 .zip 부분
extract_path = "/bast_task_set"

# 2. 먼저 /bast_task_set 경로가 없다면 생성
os.makedirs(extract_path, exist_ok=True)

# 3. unzip 명령어로 분할압축 해제
print(f"[INFO] Extracting multi-part zip with unzip: {base_zip_path}")
result = subprocess.run(["unzip", base_zip_path, "-d", extract_path], capture_output=True, text=True)

if result.returncode != 0:
    print("[ERROR] Failed to unzip base_task_set.zip:")
    print(result.stderr)
    exit(1)
else:
    print("[INFO] Successfully extracted base_task_set.zip")

# 4. TC 폴더 내의 문제번호.zip 파일들 압축 해제
tc_path = os.path.join(extract_path, "TC")

for fname in os.listdir(tc_path):
    if fname.endswith(".zip"):
        zip_file_path = os.path.join(tc_path, fname)
        problem_id = fname[:-4]  # e.g., "1001.zip" -> "1001"
        output_dir = os.path.join(tc_path, problem_id)
        os.makedirs(output_dir, exist_ok=True)

        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(output_dir)
            print(f"[INFO] Extracted {fname} to {output_dir}")


In [ ]:
# 알고리즘 문제 설명 압축 해제
%cd COCO_AI/wpc/desc/
!unzip -q desc.zip

[Errno 2] No such file or directory: 'COCO_AI/wpc/'
/COCO_AI/wpc/desc
[Errno 2] No such file or directory: 'desc/'
/COCO_AI/wpc/desc


## fixed code 생성

In [ ]:
%cd ../evaluation
!python run.py --test_data_file /base_task_set_code_fairs.json --model_path /model

## fixed code 채점

In [ ]:
!python scoring.py --output_jsonl_path pred_base_task_set_code_fairs.jsonl --base_task_dir /bast_task_set --task_detail_path /bast_task_set/task_detail.json